# Estimate Parking Capacity based on Alex's Code and Parameters

## (UNFINISHED: modified Alex's code and run with QGIS in python console, and produced all the necessary outputs)

## 0) Imports, settings and notebook utilities

In [372]:
import os
import geopandas as gpd
import geojson
import json
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import Overpass, overpassQueryBuilder
from OSMPythonTools.api import Api

In [373]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = 200

In [374]:
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
%matplotlib inline

In [375]:
from IPython.display import display, HTML

# utility function to display tables, views, etc side-by-side 
css = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(css))

In [376]:
df_lor = gpd.read_file("../data/raw/friedrichshain-kreuzberg/lor-districts.geojson")

# select only Friedrichshain-Kreuzberg
df_lor = (df_lor
          .loc[lambda d: d["BEZIRKSNAME"] == "Friedrichshain-Kreuzberg"]
          .drop(columns=["gml_id", "RAUMID"])
         )

# set as background map 
def get_lor_map():
    bezirk_map = df_lor.explore( 
        column="PLANUNGSRAUM",  # make choropleth based on "BoroName" column
        legend=True, # show legend
        legend_kwds=dict(colorbar=False), # do not use colorbar
        style_kwds=dict(opacity=0.4, fillOpacity=0.4),
        name="Friedrichshain-Kreuzberg" # name of the layer in the map
)
    return bezirk_map

## 1) OSM street and parking default parameters

In [377]:
#default width of streets (if not specified more precisely on the data object)
width_minor_street = 11
width_primary_street = 17
width_secondary_street = 15
width_tertiary_street = 13
width_service = 4
width_driveway = 2.5

#default width of parking lanes (if not specified more precisely on the data object)
width_para = 2   #parallel parking -----
width_diag = 4.5 #diagonal parking /////
width_perp = 5   #perpendicular p. |||||

#parking space length / distance per vehicle depending on parking direction
#TODO: Attention: In some calculation steps that use field calculator formulas, these values are currently still hardcoded – if needed, the formulas would have to be generated as a string using these variables
vehicle_dist_para = 5.2     #parallel parking
vehicle_dist_diag = 3.1     #diagonal parking (angle: 60 gon = 54°)
vehicle_dist_perp = 2.5     #perpendicular parking
vehicle_length = 4.4        #average vehicle length (a single vehicle, wwithout manoeuvring distance)
vehicle_width = 1.8         #average vehicle width

In [378]:
#list of attributes kept for the street layer
#Attention: Certain width specifications are also processed (fillBaseAttributes()), but they should not be specified here.
#"parking:lane:left/right:position" are new attributes for collecting the parking lane position.
#"error_output" is a new attribute to collect errors and inconsistencies
street_key_list = [
'highway',
'name',
'width_proc',
'width_proc:effective',
'surface',
'parking:lane:left',
'parking:lane:right',
'parking:lane:left:position',
'parking:lane:right:position',
'parking:lane:left:width',
'parking:lane:right:width',
'parking:lane:left:width:carriageway',
'parking:lane:right:width:carriageway',
'parking:lane:left:offset',
'parking:lane:right:offset',
'error_output'
]

## 2) Load relevant datasets

In [380]:
# parking lanes processes by Alex's script (incomplete)
qgis_python_dir = "../data/raw/friedrichshain-kreuzberg/qgis-python-parking-spot-estimation/"
# df_alex_parking_left = gpd.read_file("../data/raw/friedrichshain-kreuzberg/qgis-python-parking-spot-estimation/data/parking_lanes/parking_lanes_left.geojson")
# df_alex_parking_right = gpd.read_file("../data/raw/friedrichshain-kreuzberg/qgis-python-parking-spot-estimation/data/parking_lanes/parking_lanes_left.geojson")
# df_alex_parking_lanes = df_alex_parking_left.append(df_alex_parking_right)

In [381]:
# actual parking data in OSM
df_parking = gpd.read_file("../data/raw/friedrichshain-kreuzberg/OSM_area-FK_amenity-parking_out-geom.geojson")

In [382]:
# F-K street lanes processed by Alex's script
df_streets = gpd.read_file("../data/raw/friedrichshain-kreuzberg/qgis-python-parking-spot-estimation/data/streets_processed.geojson")

In [385]:
# F-K crossings (crubs intersections, traffic lights, driveways, pedestrian crossings) processed by Alex's script
# df_crossings = gpd.read_file("../data/raw/friedrichshain-kreuzberg/qgis-python-parking-spot-estimation/data/crossing.geojson")

In [386]:
# NK parking lanes processed by Alex's script (complete + some areas in Kreuzberg)
df_neukolln = gpd.read_file("../data/raw/neukoelln/parking_way.geojson")

In [387]:
# Bezriksamt parking spot count (incomplete)
df_bezirksamt = gpd.read_file("../data/raw/friedrichshain-kreuzberg/counted_parking-fk.geojson")

## 3) Explore parking lane datasets
 - **OSM (amenity=parking)** `OSM_area-FK_amenity-parking_out-geom.geojson` **(LABEL: OSM-PARKING)**
 - **Alex's estimation of parking lanes based on street info from OSM** `parking_lanes_left.geojson` and `parking_lanes_right.geojson` **(LABEL: ALEX-FK-PARKING)**
 - **Alex's Neukolln data (contains some LORs in Kreuzberg)** `parking_way.geojson` **(LABEL: ALEX-NK-PARKING)**
 - **Bezirksamt** `counted_parking-fk.geojson` **(LABEL: BEZIRKSAMT)**

### A) OSM-PARKING
- very incomplete info on parking lanes (mainly Reichenbergerstr.) 
- some information about parking type and orientation
- bearly no info on capacity
- a lot of missing data

In [ ]:
df_parking = (df_parking
              # drop cols with more than 100 (out of 950) null values 
              .dropna(thresh=100, axis=1)
              # only parking on lanes
              .loc[lambda d: d["parking"].isin(['street_side', 'lane'])]
             )

In [ ]:
_ = msno.bar(df_parking)

In [ ]:
df_parking.explore(m=get_lor_map())

### B) ALEX-FK-PARKING
- it covers a bit less than 2/3 of the district
- not too many missing values (at least for relevant columns)
- almost no info on parking capacity
- `capacity` and `highway:name` are strongly inversely correlated (see missingness pattern below). strange.
- most streets are `residential` and most parking spots are `parallel`

In [ ]:
_ = msno.matrix(df_alex_parking_lanes)

In [ ]:
_ = msno.heatmap(df_alex_parking_lanes)

In [ ]:
df_alex_parking_lanes.explore(m=get_lor_map())

In [ ]:
df_alex_parking_lanes.groupby(["highway", "orientation", "parking"]).size().to_frame()

In [ ]:
g = sns.catplot(x="highway", hue="orientation", col="parking",
                data=df_alex_parking_lanes, saturation=.5,
                kind="count", ci=None, aspect=1., height=8)

### C) ALEX-NK-PARKING
- very compelte data (at least on relevant columns)
- only data on Reichenbergerstr., Graefekiez, Chamissokiez
- parking spot measurements are reglamentary (see parameters above) both when the parkign spot are `estimated` or pulled from `OSM`
- parking on lanes/street only
- most streets are `residential`
- parking is mostly `parallel` and on `residential` streets
- there's a lot of `p[erpendicular` parking on `living_street` (residential but when the pedestrian has priiority)


In [ ]:
# extract F-K parts only
df_neukolln  = (df_neukolln
                .sjoin(df_lor, how="inner")
                .drop(columns="index_right")
                # .to_crs("EPSG:25833")
                .loc[:, df_neukolln.columns]
               )

In [ ]:
_ = msno.matrix(df_neukolln)

In [ ]:
df_neukolln.explore(m=get_lor_map())

In [ ]:
df_neukolln["length_per_spot"] = df_neukolln["length"] / df_neukolln["capacity"]

In [ ]:
df_neukolln.loc[:, ["parking", "orientation", "position", "capacity", "length", "length_per_spot"]].head(8)

In [ ]:
g = sns.catplot(x="orientation", y="length_per_spot", col="source:capacity",
                data=df_neukolln, saturation=.5,
                kind="bar", ci=None, aspect=1., height=8)

In [ ]:
df_neukolln.groupby(["highway", "orientation", "parking"]).size().to_frame()

In [ ]:
g = sns.catplot(x="highway", hue="orientation", col="parking",
                data=df_neukolln, saturation=.5,
                kind="count", ci=None, aspect=1., height=8)

## D) BEZIRKSAMT
- `laenge` and `anzahl` values are a lot of times off: more than 50% of the data have unrealistic measurements of parking spot lengths (> 6.1m)
- the other 50% has variables sizes: there is no info on parking orientation, so we can't know if these sizes "make sense"
- mostly types `OE_O_EIN` and `OE_KOSTENPF`

In [ ]:
df_bezirksamt.head()

In [ ]:
df_bezirksamt["length_per_spot"] = (df_bezirksamt["laenge"] / df_bezirksamt["anzahl"]).replace(np.inf, np.nan)

In [ ]:
df_bezirksamt["length_per_spot"].describe().to_frame()

In [ ]:
import numpy as np
_ = df_bezirksamt["length_per_spot"].plot(kind="hist", bins=100, figsize=(14, 8), title="Estimated parking spot lengths")

In [ ]:
g = sns.FacetGrid(df_bezirksamt, col="klassennam", col_wrap=4, height=4, aspect=1.5, ylim=(0, 100))
_ = g.map(sns.histplot, "anzahl")

## 3) Crossings, Driveways and Streets of Friedrichshain-Kreuzberg

#### Crossings
- a lot of missing values in crossings: set threshold to 15 non-null datapoints, and we end up with 25 columns (se barplot below)
- it does not matter too much since we only need the type of highway and its geometry, and those columns have no missing data
- the data seems to cover all crossing in the district 
- keep only highways of type `crossing`

#### Driveways
- same as crossings, it seems to corver all the driveways in the district
- keep only highways of type `driveways`
- get only the intersecctions (points) of driveways and streets

#### Intersections
- full network of street lanes of the district
- get the intersections of the lanes (way) in the network: usually way object start and end at intersections, but not always.


In [ ]:
_ = msno.matrix(df_crossings)

In [ ]:
df_crossings = df_crossings.dropna(thresh=15, axis=1)
_ = msno.bar(df_crossings)

In [ ]:
df_crossings.filter(regex="highway|crossing|crossing|kerb|traffic_signal").apply(lambda x: pd.Series(x.unique())).fillna("-")

In [ ]:
df_crossings = df_crossings.loc[lambda d: d["highway"] == "crossing"]

In [ ]:
crossings_map = df_crossings.explore(column="highway")
df_streets.explore(m=crossings_map, color="grey")

In [ ]:
df_services = gpd.read_file("../data/raw/friedrichshain-kreuzberg/qgis-python-parking-spot-estimation/data/service_processed.geojson")
df_services.head(2)

In [ ]:
df_driveways = df_services.loc[lambda d: d["highway"] == "service"].overlay(df_streets, how='intersection', keep_geom_type=False)

In [ ]:
df_driveways.head()

In [ ]:
df_driveways.explore()

In [ ]:
df_intersections = df_streets.overlay(df_streets, how="intersection", keep_geom_type=False)

In [ ]:
df_intersections.explore()

## 4) 
- one is not allowed to park at least 3 meters from a crossing
- one is not allowed to park in front of a driveway (driveways are usually 2.5m wide)
- there is no parking space on intersections (use street type dimensions in the parameters above)

In [ ]:
df_crossings["geometry"] = df_crossings["geometry"].buffer(9.85)

In [ ]:
d3 = df_streets.overlay(df_crossings, how="difference")

In [ ]:
d3.explore()

In [ ]:
df = gpd.read_file("/Users/nat/Downloads/data/parking_lanes.geojson")

In [ ]:
parking_map = df.explore(color="grey")

In [ ]:
df1 = gpd.read_file("/Users/nat/Downloads/data/parking_kfz.geojson")

In [ ]:
df1.explore(m=parking_map, color="red")

In [ ]:
msno.matrix(df)